In [1]:
# import necessary modules
import time
import matplotlib.pyplot as plt
import numpy as np
from keras.applications import VGG16

from keras.models import Sequential
from keras.models import Model
from keras.utils import to_categorical
from keras.layers import Dropout, Flatten, Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import imagenet_utils
from keras import backend as K
import cv2
import pdb

Using TensorFlow backend.


In [2]:
train_data_dir = '/Users/rmillin/Documents/Insight/image_reorg/fold5/train'
test_data_dir = '/Users/rmillin/Documents/Insight/image_reorg/fold5/test'


### Function for using CNN to extract image features

In [6]:


import numpy as np
import os
from os.path import isfile, join


def save_bottleneck_features(image_shape, batch_size, image_dir, filename):
    ''' 
    Saves bottleneck features for testing and training.
    '''
    print('\n Saving Bottleneck Features...')


    model = VGG16(weights="imagenet", include_top = False)

    if not(isfile(join(image_dir, filename))):
        datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range = 0,
            width_shift_range = 0,
            height_shift_range = 0,
            shear_range = 0,
            zoom_range = 0,
            fill_mode = 'nearest'
            )

        image_generator = datagen.flow_from_directory(
            image_dir,
            target_size = image_shape,
            batch_size = batch_size,
            class_mode = None,
            shuffle = False,
            )

        all_data = model.predict_generator(image_generator, verbose=1)
        np.save(open(join(image_dir, filename),'wb'), all_data)
    else:
        all_data = np.load(join(image_dir, filename))
        
    return all_data




### Function for performing logistic regression with parameter tuning

In [15]:
# function for multinomial logistic regression (with parameter tuning)

def perf_log_reg(X_train, X_val, y_train, y_val):
    
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report
    from sklearn import preprocessing
    
    s_scaler = preprocessing.MinMaxScaler()

    X_train = s_scaler.fit_transform(X_train)
    X_val = s_scaler.transform(X_val)

    # x-validated parameter search on training data

    lr = LogisticRegression(penalty='l2', multi_class='multinomial', solver='saga', max_iter=1000)
    parameters = {'C':[0.0001, 0.001, 0.1, 1, 10]}
#     parameters = {'C':[1, 10]}
    clf = GridSearchCV(lr, parameters)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()

    # evaluate on validation data

    y_true, y_pred = y_val, clf.predict(X_val)
    print(classification_report(y_true, y_pred))
    
    return y_pred, y_true


### Prepare training and validation data sets

In [13]:
# get the bottleneck features for the fully processed images

# Model inputs
batch_size = 50
epochs = 50
top_model_weights_path = 'my_model'

image_shape = (224, 224) # VGG16

# create bottleneck features for training data
image_dir = train_data_dir
filename = 'bottleneck_features_train.npy'
train_data = save_bottleneck_features(image_shape, batch_size, image_dir,filename)
train_data = np.reshape(train_data,(train_data.shape[0], 512*7*7))

# and for testing data
image_dir = test_data_dir
filename = 'bottleneck_features_test.npy'
test_data = save_bottleneck_features(image_shape, batch_size, image_dir, filename)
test_data = np.reshape(test_data,(test_data.shape[0], 512*7*7))

# # and for full set
# image_dir = '/Users/rmillin/Documents/Insight/image_reorg/fullset'
# filename = 'bottleneck_features'
# all_data = save_bottleneck_features(image_shape, batch_size, image_dir, filename)
# all_data = np.reshape(all_data,(all_data.shape[0], 512*7*7))



 Saving Bottleneck Features...

 Saving Bottleneck Features...


### Do the classifier optimization, training, and validation

In [16]:

n_classes = 5
n_per_class_train = int(train_data.shape[0]/n_classes)
n_per_class_test = int(test_data.shape[0]/n_classes)
train_labels = []
test_labels = []
for this_class in range(n_classes):
    train_labels = train_labels + [this_class] * n_per_class_train
    test_labels = test_labels + [this_class] * n_per_class_test
  # labels
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

# # set aside validation data
# # because of rotations, groups of 4 images all belong to the same original image; preserve this
# X_train, X_val, y_train, y_val = train_test_split(all_data, all_labels, test_size=0.2, stratify=all_labels)

X_train = train_data
X_val = test_data

y_train = train_labels
y_val = test_labels

y_pred, y_true = perf_log_reg(X_train, X_val, y_train, y_val)

np.save(open(os.path.join(test_data_dir,'pred_nn_filt.npy'),'wb'), y_pred)
np.save(open(os.path.join(test_data_dir,'act_nn_filt.npy'),'wb'), y_true)


/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Best parameters set found on development set:

{'C': 0.1}

Grid scores on development set:

0.382 (+/-0.129) for {'C': 0.0001}
0.445 (+/-0.108) for {'C': 0.001}
0.545 (+/-0.074) for {'C': 0.1}
0.529 (+/-0.051) for {'C': 1}
0.532 (+/-0.056) for {'C': 10}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       0.43      0.41      0.42        32
          1       0.80      0.88      0.84        32
          2       0.51      0.69      0.59        32
          3       0.76      0.69      0.72        32
          4       0.74      0.53      0.62        32

avg / total       0.65      0.64      0.64       160



### Train the optimized classifier on the full dataset for use in the app (where images will be novel and not from either set)

In [10]:
# use all of this data for the model for classifying new images

import pickle
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

image_dir = '/Users/rmillin/Documents/Insight/image_reorg/fullset'

n_classes = 5

all_data = save_bottleneck_features(image_shape, batch_size, image_dir, 'bottleneck_features.npy')
all_data = np.reshape(all_data,(all_data.shape[0], 512*7*7))

n_total = all_data.shape[0]
n_per_class = int(n_total/n_classes)
all_labels = []
for this_class in range(n_classes):
    all_labels = all_labels + [this_class] * n_per_class
  # labels
all_labels = np.array(all_labels)

s_scaler = preprocessing.MinMaxScaler()
all_data = s_scaler.fit_transform(all_data)

clf = LogisticRegression(penalty='l2', C=0.1, multi_class='multinomial', solver='saga').fit(all_data, all_labels)
# clf = LogisticRegression(penalty='l1', C=1, multi_class='multinomial', solver='saga').fit(all_data, all_labels)
perf = clf.score(all_data, all_labels)
print(perf)




 Saving Bottleneck Features...
Found 1020 images belonging to 5 classes.


KeyboardInterrupt: 

In [11]:
filename = 'finalized_model.sav'
pickle.dump(clf, open(join('/Users/rmillin/Documents/Insight/animal-tracks/mvpapp/webapp/static/data',filename), 'wb'))
filename = 'scaler.sav'
pickle.dump(s_scaler, open(join('/Users/rmillin/Documents/Insight/animal-tracks/mvpapp/webapp/static/data',filename), 'wb'))


### Assess performance on other pipelines: CNN features for unfiltered images, pixel values of unfiltered images, and pixel values of filtered images

In [18]:
# get the bottleneck features for the unfiltered images

# Model inputs
batch_size = 50
epochs = 50
top_model_weights_path = 'my_model'

image_shape = (224, 224) # VGG16

# create bottleneck features for training data
image_dir = '/Users/rmillin/Documents/Insight/image_reorg/fold5_unfilt/train'
filename = 'bottleneck_features_unfilt_train.npy'
train_data = save_bottleneck_features(image_shape, batch_size, image_dir, filename)
train_data = np.reshape(train_data,(train_data.shape[0], 512*7*7))

# and for testing data
image_dir = '/Users/rmillin/Documents/Insight/image_reorg/fold5_unfilt/test'
filename = 'bottleneck_features_unfilt_test.npy'
test_data = save_bottleneck_features(image_shape, batch_size, image_dir, filename)
test_data = np.reshape(test_data,(test_data.shape[0], 512*7*7))



 Saving Bottleneck Features...
Found 840 images belonging to 5 classes.
17/17 [==============================] - 691s 41s/step

 Saving Bottleneck Features...
Found 160 images belonging to 5 classes.
4/4 [==============================] - 126s 31s/step


In [19]:
# get performance on CNN features for unfiltered images

X_train = train_data
X_val = test_data

y_train = train_labels
y_val = test_labels

y_pred, y_true = perf_log_reg(X_train, X_val, y_train, y_val)

np.save(open(os.path.join(image_dir,'pred_cnn_unfilt.npy'),'wb'), y_pred)
np.save(open(os.path.join(image_dir,'act_cnn_unfilt.npy'),'wb'), y_true)


/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Best parameters set found on development set:

{'C': 0.1}

Grid scores on development set:

0.318 (+/-0.096) for {'C': 0.0001}
0.387 (+/-0.122) for {'C': 0.001}
0.480 (+/-0.064) for {'C': 0.1}
0.464 (+/-0.090) for {'C': 1}
0.463 (+/-0.101) for {'C': 10}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       0.24      0.25      0.25        32
          1       0.50      0.53      0.52        32
          2       0.21      0.22      0.22        32
          3       0.62      0.62      0.62        32
          4       0.46      0.41      0.43        32

avg / total       0.41      0.41      0.41       160



In [20]:
# also get accuracy for RGB values for filtered images

import pdb
import cv2

train_data_dir = '/Users/rmillin/Documents/Insight/image_reorg/fold5/train'
test_data_dir = '/Users/rmillin/Documents/Insight/image_reorg/fold5/test'

animals = ['bear','canine','feline','hooved','others']
    
train_data=np.empty([0, 224**2])
test_data=np.empty([0, 224**2])
for animal in animals:
    for name in os.listdir(os.path.join(train_data_dir,animal)):
        try:
            img = cv2.imread(os.path.join(train_data_dir,animal,name))
            train_data = np.append(train_data,np.matrix(img[:,:,0].flatten()),axis=0)
        except:
#            pdb.set_trace()
            print(name)

    for name in os.listdir(os.path.join(test_data_dir,animal)):
        try:
            img = cv2.imread(os.path.join(test_data_dir,animal,name))
            test_data = np.append(test_data,np.matrix(img[:,:,0].flatten()),axis=0)
        except:
    #        pdb.set_trace()
            print(name)




.DS_Store
.DS_Store
.DS_Store
.DS_Store
.DS_Store


In [23]:
# logistic regression classifier on filtered RGB

X_train = train_data
X_val = test_data

y_pred, y_true = perf_log_reg(X_train, X_val, y_train, y_val)

np.save(open(os.path.join(test_data_dir,'pred_rgb_filt.npy'),'wb'), y_pred)
np.save(open(os.path.join(test_data_dir,'act_rgb_filt.npy'),'wb'), y_true)




/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/skle

Best parameters set found on development set:

{'C': 0.0001}

Grid scores on development set:

0.338 (+/-0.084) for {'C': 0.0001}
0.323 (+/-0.074) for {'C': 0.001}
0.325 (+/-0.096) for {'C': 0.1}
0.327 (+/-0.105) for {'C': 1}
0.329 (+/-0.099) for {'C': 10}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        32
          1       0.32      0.38      0.35        32
          2       0.33      0.38      0.35        32
          3       0.00      0.00      0.00        32
          4       0.23      0.62      0.34        32

avg / total       0.18      0.28      0.21       160



/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:

# performance on raw rgb values

train_data_dir = '/Users/rmillin/Documents/Insight/image_reorg/fold5_unfilt/train'
test_data_dir = '/Users/rmillin/Documents/Insight/image_reorg/fold5_unfilt/test'

animals = ['bear','canine','feline','hooved','others']
    
train_data=np.empty([0, 224**2])
test_data=np.empty([0, 224**2])
for animal in animals:
    for name in os.listdir(os.path.join(train_data_dir,animal)):
        try:
            img = cv2.imread(os.path.join(train_data_dir,animal,name))
            train_data = np.append(train_data,np.matrix(img[:,:,0].flatten()),axis=0)
        except:
#            pdb.set_trace()
            print(name)

    for name in os.listdir(os.path.join(test_data_dir,animal)):
        try:
            img = cv2.imread(os.path.join(test_data_dir,animal,name))
            test_data = np.append(test_data,np.matrix(img[:,:,0].flatten()),axis=0)
        except:
    #        pdb.set_trace()
            print(name)




In [26]:
# logistic regression on raw RGB

X_train = train_data
X_val = test_data

y_pred, y_true = perf_log_reg(X_train, X_val, y_train, y_val)

np.save(open(os.path.join(test_data_dir,'pred_rgb_unfilt.npy'),'wb'), y_pred)
np.save(open(os.path.join(test_data_dir,'act_rgb_unfilt.npy'),'wb'), y_true)


/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Applications/anaconda/envs/insight/lib/python3.6/site-packages/skle

Best parameters set found on development set:

{'C': 0.001}

Grid scores on development set:

0.346 (+/-0.056) for {'C': 0.0001}
0.352 (+/-0.095) for {'C': 0.001}
0.338 (+/-0.057) for {'C': 0.1}
0.338 (+/-0.062) for {'C': 1}
0.338 (+/-0.062) for {'C': 10}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       0.31      0.12      0.18        32
          1       0.55      0.50      0.52        32
          2       0.47      0.50      0.48        32
          3       0.00      0.00      0.00        32
          4       0.19      0.44      0.26        32

avg / total       0.30      0.31      0.29       160



In [ ]:

from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import numpy as np
import os
from os.path import isfile, join

# Data inputs

# Model inputs
batch_size = 50
epochs = 50
top_model_weights_path = 'my_model'

image_shape = (224, 224) # VGG16

# train_data_dir = '/Users/rmillin/Documents/Insight/animal-tracks/mvpapp/webapp/static/images/train'
# test_data_dir = '/Users/rmillin/Documents/Insight/animal-tracks/mvpapp/webapp/static/images/test'
# train_data_dir = '/Users/rmillin/Documents/Insight/animal-tracks/multiclass/train_grayscale'
# test_data_dir = '/Users/rmillin/Documents/Insight/animal-tracks/multiclass/test_grayscale'


def save_bottleneck_features(image_shape, batch_size, test_data_dir, train_data_dir):
    ''' 
    Saves bottleneck features for testing and training.
    '''
    print('\n Saving Bottleneck Features...')


    model = VGG16(weights="imagenet", include_top = False)

#     datagen = ImageDataGenerator(
#         rescale=1./255,
#         rotation_range = 10,
#         width_shift_range = 0.2,
#         height_shift_range = 0.2,
#         shear_range = 0,
#         zoom_range = 0.2,
#         fill_mode = 'nearest'
#         )
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range = 0,
        width_shift_range = 0,
        height_shift_range = 0,
        shear_range = 0,
        zoom_range = 0,
        fill_mode = 'nearest'
        )

    train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size = image_shape,
        batch_size = batch_size,
        class_mode = None,
        shuffle = False,
        save_prefix='aug'  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
        )

    if not(isfile('gray_all_bottleneck_features_train.npy')):
        train_data = model.predict_generator(train_generator, verbose=1)
        np.save(open(join(train_data_dir, 'bottleneck_features_train.npy'),'wb'), train_data)
    else:
        train_data = np.load(join(train_data_dir, 'bottleneck_features_train.npy'))
        
    test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size = image_shape,
        batch_size = batch_size,
        class_mode = None,
        shuffle = False,
        save_prefix='aug'  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
        )
            
    if not(isfile(join(test_data_dir, 'bottleneck_features_test.npy'))):
        test_data = model.predict_generator(test_generator, verbose=1)
        np.save(open(join(test_data_dir, 'bottleneck_features_test.npy'),'wb'), test_data)
    else:
        test_data = np.load(join(test_data_dir, 'bottleneck_features_test.npy'))
        
    # with a Sequential model
    layer_name = 'block3_pool'
    intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)

    train_activations = intermediate_layer_model.predict_generator(train_generator)
    test_activations = intermediate_layer_model.predict_generator(test_generator)

    
    train_y = to_categorical(train_generator.classes)
    test_y = to_categorical(test_generator.classes)
    train_labels = train_generator.class_indices
    np.save(open(join(train_data_dir, 'train_y.npy'),'wb'), train_y)
    np.save(open(join(test_data_dir, 'test_y.npy'),'wb'), test_y)
        
    return train_data, test_data, train_y, train_labels, test_y, train_activations, test_activations

def train_top_model(train_data, train_y, test_data, test_y, n_classes, top_model_weights_path):
    ''' 
    Train top layer with bottleneck features as input
    '''

    print('\n Training the FC Layers...')
   
    model = Sequential()
    model.add(Flatten(input_shape = train_data.shape[1:]))
    model.add(Dense(2056, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1028, activation = 'relu'))
    model.add(Dense(n_classes, activation = 'softmax'))

    opt = optimizers.SGD(lr = 1.0e-4, momentum=0.9)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])

    checkpointer = ModelCheckpoint(filepath='model.best.hdf5', verbose=1, save_best_only=False)
   
    model.fit(train_data, train_y,
             epochs=epochs,
             batch_size=batch_size,
             validation_data = [test_data, test_y],
             callbacks = [checkpointer])

    model.save_weights(top_model_weights_path)

    return model  

if (not isfile(join(train_data_dir, 'bottleneck_features_train.npy'))) or (not isfile(join(test_data_dir,'bottleneck_features_test.npy'))):
    train_data, test_data, train_y, train_labels, test_y, train_activations, test_activations = \
        save_bottleneck_features(image_shape, \
        batch_size, test_data_dir, train_data_dir)
else:
    train_data = np.load(join(train_data_dir,'bottleneck_features_train.npy'))
    test_data = np.load(join(test_data_dir, 'bottleneck_features_test.npy'))
    train_y = np.load(train_data_dir,'train_y.npy')
    test_y = np.load(test_data_dir, 'test_y.npy') 
    
print(train_data.shape)
print(test_data.shape)



 Saving Bottleneck Features...
Found 840 images belonging to 5 classes.
 4/17 [======>.......................] - ETA: 10:54